In [16]:
import pandas as pd

df = pd.read_csv("hotel_bookings.csv")

# تبدیل 'NULL'ها (اگر به صورت رشته ذخیره شده‌اند) به NaN واقعی
df = df.replace('NULL', pd.NA)

# پر کردن مقادیر گمشده عددی با صفر (ساده‌ترین راهکار)
df['children'] = df['children'].fillna(0)
df['company'] = df['company'].fillna(0)
df['agent'] = df['agent'].fillna(0)

# حذف ستون‌هایی که داده‌های زیادی از دست رفته دارند یا در این مرحله لازم نیستند
df = df.drop(['company', 'agent', 'reservation_status', 'reservation_status_date'], axis=1)

# حذف سطرهایی که هنوز داده گمشده‌ای دارند (مثلاً در Country)
df = df.dropna()

In [17]:
# لیست ستون‌های طبقه‌بندی (که باید به 0 و 1 تبدیل شوند)
categorical_features = [
    'hotel', 'meal', 'country', 'market_segment', 'distribution_channel', 
    'reserved_room_type', 'assigned_room_type', 'deposit_type', 'customer_type','arrival_date_month'
]

# اعمال One-Hot Encoding
df_encoded = pd.get_dummies(df, columns=categorical_features, drop_first=True)

# حالا تمام ستون‌های متنی به ستون‌های 0 و 1 تبدیل شده‌اند.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# متغیر هدف: 'is_canceled'
X = df_encoded.drop('is_canceled', axis=1)
y = df_encoded['is_canceled']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model_rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1) 
# n_jobs=-1 از تمام هسته‌های CPU استفاده می‌کند و سرعت را بالا می‌برد.

model_rf.fit(X_train, y_train)

# ارزیابی
y_pred = model_rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy of Random Forest (After Preprocessing): {accuracy * 100:.2f}%")

Accuracy of Random Forest (After Preprocessing): 86.56%


In [19]:
# نمایش اهمیت ویژگی‌ها
feature_importances = pd.Series(model_rf.feature_importances_, index=X.columns).sort_values(ascending=False)
print("\n--- 5 Feature Importances ---")
print(feature_importances.head(5))


--- 5 Feature Importances ---
deposit_type_Non Refund    0.146029
lead_time                  0.126163
country_PRT                0.091133
arrival_date_year          0.061035
hotel_Resort Hotel         0.056324
dtype: float64


پیشبینی با استفاده از pipeline

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
# ابزارهای مورد نیاز برای Pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

In [2]:
# ===================================================
# مرحله ۱: بارگذاری و تعریف ستون‌ها
# ===================================================
df = pd.read_csv("hotel_bookings.csv")

df = df.drop(['company', 'agent', 'reservation_status', 'reservation_status_date'], axis=1)

X = df.drop('is_canceled', axis=1)
y = df['is_canceled']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [4]:
# ===================================================
# مرحله ۲: ساخت Pipeline (جایگزین تمیزکاری دستی)
# ===================================================
# تعریف ستون‌های عددی و طبقه‌بندی (بر اساس نوع داده‌های خام)
numeric_features = ['lead_time', 'arrival_date_year', 'arrival_date_week_number', 'arrival_date_day_of_month', 
                    'stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'children', 'babies', 
                    'is_repeated_guest', 'previous_cancellations', 'previous_bookings_not_canceled', 
                    'booking_changes', 'days_in_waiting_list', 'adr', 'required_car_parking_spaces', 
                    'total_of_special_requests'] 
categorical_features = ['hotel', 'meal', 'country', 'market_segment', 'distribution_channel', 
                        'reserved_room_type', 'assigned_room_type', 'deposit_type', 'customer_type', 
                        'arrival_date_month']

# 1. Pipeline برای ستون‌های عددی (پر کردن NaN با صفر)
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value=0))
])

# 2. Pipeline برای ستون‌های طبقه‌بندی (پر کردن NaN با 'missing' و One-Hot)
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')), # پر کردن NaN (مانند Country)
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop='first'))
])

# 3. ترکیب Transformerها
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='drop' 
)

# 4. Pipeline نهایی (آماده‌سازی + مدل)
final_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1))
])

In [5]:
# ===================================================
# مرحله ۳: آموزش و پیش‌بینی (فقط دو خط!)
# ===================================================
final_pipeline.fit(X_train, y_train)

y_pred = final_pipeline.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of Random Forest (using Pipeline): {accuracy * 100:.2f}%")

Accuracy of Random Forest (using Pipeline): 86.10%


/home/codespace/.local/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [2, 6] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [7]:
required_columns = X_train.columns.tolist()

# ساخت داده جدید با همه ستون‌ها
new_reservation_data = {
    'hotel': ['City Hotel'], 
    'lead_time': [10], 
    'arrival_date_year': [2026],
    'arrival_date_month': ['December'],
    'arrival_date_week_number': [50],
    'arrival_date_day_of_month': [15],
    'stays_in_weekend_nights': [0], 
    'stays_in_week_nights': [5], 
    'adults': [2], 
    'children': [0], 
    'babies': [0], 
    'meal': ['BB'], 
    'country': ['USA'], 
    'market_segment': ['Online TA'],
    'distribution_channel': ['TA/TO'],
    'is_repeated_guest': [0],
    'previous_cancellations': [0],
    'previous_bookings_not_canceled': [0],
    'reserved_room_type': ['A'],
    'assigned_room_type': ['A'],
    'booking_changes': [0],
    'deposit_type': ['No Deposit'],
    'days_in_waiting_list': [0],
    'customer_type': ['Transient'],
    'adr': [95.0],
    'required_car_parking_spaces': [0],
    'total_of_special_requests': [1]
}

new_df_raw = pd.DataFrame(new_reservation_data)

prediction = final_pipeline.predict(new_df_raw)

result = prediction[0]
print(result)

1
